# import

In [1]:
import numpy as np
import pandas as pd

# CUSIP

In [2]:
# CUSIP 
cusips = ['91282CAX9', '91282CBA8', '91282CAZ4',
          '91282CAY7', '91282CAV3', '912810ST6', '912810SS8']
n = len(cusips)
num_prices = 10000
num_trades = 10
num_marketdata = 10000
num_inquiries = 10

# helper functions

In [3]:
# Convert a double -> 1/256th precision
def convert_price256(x):
    return (int(x * 256) / 256)

In [4]:
# Convert price data from double -> string format
def convert_price(x):
    x_100 = int(x)
    x = 32 * (x - x_100)
    x_32 = int(x)
    x -= x_32
    x_256 = int(x * 8)
    
    if x_256 == 4:
        s = '00' + str(x_32) + '+'
    else:
        s = '00' + str(x_32) + str(x_256)
    s = s[-3:]
    s = str(x_100) + '-' + s
    return s

# Generate input data files

## Generate prices.txt

In [17]:
# Generate prices.txt
def get_prices():
    prices = pd.DataFrame(index=[i + 1 for i in range(num_prices * n)],
                          columns=['productID', 'bid', 'offer'])

    prices_99_101 = np.random.uniform(99, 101, num_prices * n)
    prices_99_101 = [convert_price256(x) for x in prices_99_101]
    spread = [1 / 128, 1 / 64] * int(num_prices * n / 2)

    prices['productID'] = [cusip for i in range(num_prices) for cusip in cusips]
    prices['bid'] = prices_99_101
    prices['offer'] = spread
    prices['offer'] += prices['bid']
    prices['bid'] = prices['bid'].apply(convert_price)
    prices['offer'] = prices['offer'].apply(convert_price)

    np.savetxt('./input/prices.txt', prices.values, fmt='%s', delimiter=" ")

    return prices

In [18]:
get_prices()

,productID,bid,offer
1,91282CAX9,99-055,99-057
2,91282CBA8,100-261,100-265
3,91282CAZ4,99-065,99-067
4,91282CAY7,99-295,99-301
5,91282CAV3,99-225,99-227
...,...,...,...
69996,91282CAZ4,99-095,99-101
69997,91282CAY7,99-16+,99-166
69998,91282CAV3,100-15+,100-160
69999,912810ST6,100-297,100-301


## Generate trades.txt

In [37]:
# Generate trades.txt
def get_trades():
    trades = pd.DataFrame(index = [i+1 for i in range(num_trades * n)],
                            columns = ['productID','traderID','price','books','quantity','side'])

    trades['productID'] = [cusip for i in range(num_trades) for cusip in cusips]
    
    traderID = np.random.uniform(0, 999999999, num_trades * n)
    traderID = [str(int(x)).zfill(10) for x in traderID]
    trades['traderID'] = traderID

    prices_99_101 = np.random.uniform(99, 101, num_trades * n)
    prices_99_101 = [convert_price256(x) for x in prices_99_101]
    trades['price'] = prices_99_101
    trades['price'] = trades['price'].apply(convert_price)

    books = ['TRSY1', 'TRSY2', 'TRSY3'] * int(num_trades * n / 3 + 10)
    trades['books'] = books[:len(trades)]

    quantity = [1000000, 2000000, 3000000, 4000000, 5000000] * int(num_trades * n / 5)
    trades['quantity'] = quantity

    side = ['BUY', 'SELL'] * int(num_trades * n / 2)
    trades['side'] = side

    np.savetxt('./input/trades.txt', trades.values, fmt='%s', delimiter=" ")
    
    return trades

In [38]:
get_trades()

,productID,traderID,price,books,quantity,side
1,91282CAX9,0004752902,100-13+,TRSY1,1000000,BUY
2,91282CBA8,0845557012,100-272,TRSY2,2000000,SELL
3,91282CAZ4,0764909875,99-303,TRSY3,3000000,BUY
4,91282CAY7,0361049172,100-277,TRSY1,4000000,SELL
5,91282CAV3,0578159534,100-030,TRSY2,5000000,BUY
...,...,...,...,...,...,...
66,91282CAZ4,0464437015,99-192,TRSY3,1000000,SELL
67,91282CAY7,0221326772,100-27+,TRSY1,2000000,BUY
68,91282CAV3,0393535351,99-086,TRSY2,3000000,SELL
69,912810ST6,0044305628,99-311,TRSY3,4000000,BUY


## Generate marketdata.txt

In [32]:
# Generate marketdata.txt
def get_marketdata():
    marketdata = pd.DataFrame(index = [i+1 for i in range(num_marketdata * n)],
                            columns = ['productID','price','size','side'])
    
    mid = np.random.uniform(99, 101, int(num_marketdata * n / 2))
    mid = [convert_price256(x) for x in mid]

    spread = [1/128, 2/128, 3/128, 4/128, 3/128, 2/128] * (int(len(mid) / 6) + 10)
    spread = spread[:len(mid)]

    prices = [[x[0]-x[1]/2, x[0]+x[1]/2] for x in zip(mid, spread)]
    prices = [convert_price(i) for sub in prices for i in sub]

    marketdata['productID'] = [cusip for i in range(num_marketdata) for cusip in cusips]

    marketdata['price'] = prices

    size = [1000000, 1000000, 2000000, 2000000, 3000000, 3000000, 4000000, 4000000, 5000000, 5000000] * int(len(marketdata) / 10)
    marketdata['size'] = size

    marketdata['side'] = ['BID', 'OFFER'] * int(len(marketdata) / 2)

    np.savetxt('./input/marketdata.txt', marketdata.values, fmt='%s', delimiter=" ")
    
    return marketdata

In [33]:
get_marketdata()

,productID,price,size,side
1,91282CAX9,99-102,1000000,BID
2,91282CBA8,99-10+,1000000,OFFER
3,91282CAZ4,100-061,2000000,BID
4,91282CAY7,100-065,2000000,OFFER
5,91282CAV3,99-14+,3000000,BID
...,...,...,...,...
69996,91282CAZ4,100-06+,3000000,OFFER
69997,91282CAY7,99-153,4000000,BID
69998,91282CAV3,99-155,4000000,OFFER
69999,912810ST6,100-013,5000000,BID


## Generate inquiries.txt

In [7]:
# Generate inquiries.txt
def get_inquiries():
    inquiries = pd.DataFrame(index = [i+1 for i in range(num_inquiries * n)],
                            columns = ['inquiryID','productID','side','size','price','state'])

    inquiryID = np.random.uniform(0, 999999999, num_inquiries * n)
    inquiryID = [str(int(x)).zfill(10) for x in inquiryID]
    inquiries['inquiryID'] = inquiryID
    
    inquiries['productID'] = [cusip for i in range(num_inquiries) for cusip in cusips]

    side = ['BUY', 'SELL'] * int(len(inquiries) / 2)
    inquiries['side'] = side

    size = [1000000, 2000000, 3000000, 4000000, 5000000] * int(len(inquiries) / 5)
    inquiries['size'] = size
    
    prices_99_101 = np.random.uniform(99, 101, num_inquiries * n)
    prices_99_101 = [convert_price256(x) for x in prices_99_101]
    inquiries['price'] = prices_99_101
    inquiries['price'] = inquiries['price'].apply(convert_price)

    inquiries['state'] = 'RECEIVED'

    np.savetxt('./input/inquiries.txt', inquiries.values, fmt='%s', delimiter=" ")
    
    return inquiries

In [8]:
get_inquiries()

,inquiryID,productID,side,size,price,state
1,0605041295,91282CAX9,BUY,1000000,100-305,RECEIVED
2,0866316684,91282CBA8,SELL,2000000,100-207,RECEIVED
3,0409942552,91282CAZ4,BUY,3000000,100-265,RECEIVED
4,0871770781,91282CAY7,SELL,4000000,100-227,RECEIVED
5,0238817081,91282CAV3,BUY,5000000,99-225,RECEIVED
...,...,...,...,...,...,...
66,0591523340,91282CAZ4,SELL,1000000,99-067,RECEIVED
67,0244832682,91282CAY7,BUY,2000000,99-152,RECEIVED
68,0791698191,91282CAV3,SELL,3000000,100-021,RECEIVED
69,0026272071,912810ST6,BUY,4000000,99-01+,RECEIVED
